## Exercise 1: `mersenne_numbers`

A Mersenne number is any number that can be written as $2^p - 1$ for some $p$. For example, 3 is a Mersenne number ($2^2 - 1$) as is 31 ($2^5 - 1$). We will see later on that it is easy to test if Mersenne numbers are prime.

Write a function that accepts an exponent $p$ and returns the corresponding Mersenne number.

In [2]:
def mersenne_number(p):
    return ((2**p) -1)

Mersenne numbers can only be prime if their exponent, $p$, is prime. Make a list of the Mersenne numbers for all primes $p$ between 3 and 65 (there should be 17 of them).

Hint: It may be useful to modify the `is_prime` and `get_primes` functions from [the Program Flow notebook](https://www.codecademy.com/forum_questions/51f239449c4e9d4e3c001f43) for use in this problem.

In [12]:
def is_prime(number):
    if number <= 1:
        return False
    
    for factor in range(2, number):
        if number % factor == 0:
            return False

    return True


def print_primes(n):
    my_lst = []
    for number in range(3, n+1):
        if is_prime(number):
            my_lst.append(number)
    return my_lst

In [13]:
my_lst = print_primes(65)    #This prints the prime numbers between 3 and  65
print(my_lst)

[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61]


In [14]:
another_lst = []
for i in my_lst:
    another_lst.append(mersenne_number(i))
print(another_lst)

[7, 31, 127, 2047, 8191, 131071, 524287, 8388607, 536870911, 2147483647, 137438953471, 2199023255551, 8796093022207, 140737488355327, 9007199254740991, 576460752303423487, 2305843009213693951]


The next cell shows a dummy solution, a list of 17 sevens. Alter the next cell to make use of the functions you've defined above to create the appropriate list of Mersenne numbers.

In [15]:
mersennes = [mersenne_number(i) for i in my_lst]
print(mersennes)

[7, 31, 127, 2047, 8191, 131071, 524287, 8388607, 536870911, 2147483647, 137438953471, 2199023255551, 8796093022207, 140737488355327, 9007199254740991, 576460752303423487, 2305843009213693951]


## Exercise 2: `lucas_lehmer`

We can test if a Mersenne number is prime using the [Lucas-Lehmer test](https://en.wikipedia.org/wiki/Lucas%E2%80%93Lehmer_primality_test). First let's write a function that generates the sequence used in the test. Given a Mersenne number with exponent $p$, the sequence can be defined as

$$ n_0 = 4 $$
$$ n_i = (n_{i-1}^2 - 2) mod (2^p - 1) $$

Write a function that accepts the exponent $p$ of a Mersenne number and returns the Lucas-Lehmer sequence up to $i = p - 2$ (inclusive). Remember that the [modulo operation](https://en.wikipedia.org/wiki/Modulo_operation) is implemented in Python as `%`.

In [16]:
def lucas_lehmer(p):
    ll_seq = [4]
    m = 2**p - 1
    for i in range(1,p-1):
        n_i = ((ll_seq[i-1])** 2 - 2) % m
        ll_seq.append(n_i)
    return ll_seq

In [17]:
print(lucas_lehmer(17))

[4, 14, 194, 37634, 95799, 119121, 66179, 53645, 122218, 126220, 70490, 69559, 99585, 78221, 130559, 0]


## Exercise 3: `mersenne_primes`

For a given Mersenne number with exponent $p$, the number is prime if the Lucas-Lehmer series is 0 at position $p-2$. Write a function that tests if a Mersenne number with exponent $p$ is prime. Test if the Mersenne numbers with prime $p$ between 3 and 65 (i.e. 3, 5, 7, ..., 61) are prime. Your final answer should be a list of tuples consisting of `(Mersenne exponent, 0)` (or `1`) for each Mersenne number you test, where `0` and `1` are replacements for `False` and `True` respectively.

_HINT: The `zip` function is useful for combining two lists into a list of tuples_

In [18]:
def ll_prime(p):
    if lucas_lehmer(p)[-1] == 0:
        return 1
    return 0

In [19]:
list(zip((i for i in my_lst),(ll_prime(i) for i in my_lst)))

[(3, 1),
 (5, 1),
 (7, 1),
 (11, 0),
 (13, 1),
 (17, 1),
 (19, 1),
 (23, 0),
 (29, 0),
 (31, 1),
 (37, 0),
 (41, 0),
 (43, 0),
 (47, 0),
 (53, 0),
 (59, 0),
 (61, 1)]